In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import shutil
sys.path.append('../script')
from utils import *

In [2]:
annotation = pd.read_csv('../data/train_annotation/_annotation.csv')

In [3]:
annotation.sample(10)

,im_name,x_min,y_min,x_max,y_max,class,models
1508,64684032.jpg,7.0,14.0,294.0,174.0,1,Audi A5 Coupe 2012
556,292880384.jpg,42.0,55.0,264.0,152.0,1,Chevrolet Corvette ZR1 2012
493,250216448.jpg,5.0,41.0,194.0,150.0,1,Buick Rainier SUV 2007
1680,735772672.jpg,17.0,74.0,624.0,425.0,1,Hyundai Genesis Sedan 2012
269,134742016.jpg,8.0,13.0,622.0,267.0,1,BMW ActiveHybrid 5 Sedan 2012
803,389152768.jpg,2.0,24.0,255.0,138.0,1,Chevrolet Silverado 1500 Extended Cab 2012
1126,533659648.jpg,42.0,8.0,465.0,280.0,1,FIAT 500 Convertible 2012
233,127139840.jpg,17.0,129.0,632.0,378.0,1,Audi S4 Sedan 2007
28,1008467968.jpg,18.0,57.0,633.0,427.0,1,Toyota Camry Sedan 2012
2022,867041280.jpg,51.0,127.0,599.0,384.0,1,Mercedes-Benz 300-Class Convertible 1993


In [4]:
annotation.describe()

,x_min,y_min,x_max,y_max,class
count,2345.000000,2345.00000,2345.000000,2345.000000,2350.000000
mean,59.340299,99.60597,570.800000,369.890405,0.891064
std,82.315349,105.97295,436.464044,283.013862,0.311625
min,0.000000,0.00000,0.000000,0.000000,0.000000
25%,12.000000,29.00000,291.000000,192.000000,1.000000
50%,35.000000,72.00000,539.000000,334.000000,1.000000
75%,74.000000,133.00000,688.000000,455.000000,1.000000
max,1048.000000,1189.00000,4758.000000,2994.000000,1.000000


In [5]:
annotation[annotation["x_min"].isnull()]

,im_name,x_min,y_min,x_max,y_max,class,models
390,18022400.jpg,NaN,NaN,NaN,NaN,1,Acura TL Type-S 2008
734,37027840.jpg,NaN,NaN,NaN,NaN,1,Acura ZDX Hatchback 2012
754,37421056.jpg,NaN,NaN,NaN,NaN,1,Acura ZDX Hatchback 2012
1623,716701696.jpg,NaN,NaN,NaN,NaN,1,Hyundai Veracruz SUV 2012
1961,835125248.jpg,NaN,NaN,NaN,NaN,1,Land Rover LR2 SUV 2012


In [6]:
annotation_clean = annotation.dropna(subset=["x_min"])
annotation_clean

,im_name,x_min,y_min,x_max,y_max,class,models
0,1000407040.jpg,7.0,26.0,392.0,225.0,1,Toyota Sequoia SUV 2012
1,1000472576.jpg,27.0,62.0,519.0,298.0,1,Toyota Sequoia SUV 2012
2,1000669184.jpg,6.0,62.0,499.0,286.0,1,Toyota Sequoia SUV 2012
3,1000800256.jpg,68.0,16.0,450.0,222.0,1,Toyota Sequoia SUV 2012
4,1001324544.jpg,33.0,56.0,408.0,294.0,1,Toyota Sequoia SUV 2012
...,...,...,...,...,...,...,...
2345,993853440.jpg,54.0,81.0,915.0,484.0,1,Suzuki SX4 Sedan 2012
2346,994246656.jpg,46.0,127.0,675.0,481.0,1,Suzuki SX4 Sedan 2012
2347,999489536.jpg,80.0,198.0,940.0,583.0,1,Toyota Sequoia SUV 2012
2348,999686144.jpg,77.0,124.0,528.0,297.0,1,Toyota Sequoia SUV 2012


In [7]:
w,h = (256, 256)
images_normalized, labels_normalized = normalize_images_and_boxes(annotation_clean, (w, h), "../data/all_images/")

In [8]:
labels_normalized2 = pd.concat([pd.DataFrame(labels_normalized, columns=["x_min", "y_min", "x_max", "y_max"], index = annotation_clean.index), annotation_clean[["im_name", "class", "models"]]], axis=1)[["im_name", "x_min", "y_min", "x_max", "y_max", "class", "models"]]
labels_normalized2["x_mid"] = (labels_normalized2["x_min"] + labels_normalized2["x_max"])/2
labels_normalized2["y_mid"] = (labels_normalized2["y_min"] + labels_normalized2["y_max"])/2
labels_normalized2["w"] = labels_normalized2["x_max"] - labels_normalized2["x_min"]
labels_normalized2["h"] = labels_normalized2["y_max"] - labels_normalized2["y_min"]
labels_normalized2

,im_name,x_min,y_min,x_max,y_max,class,models,x_mid,y_mid,w,h
0,1000407040.jpg,4.480000,26.624000,250.880000,230.400000,1,Toyota Sequoia SUV 2012,127.680000,128.512000,246.400000,203.776000
1,1000472576.jpg,12.020870,48.987654,231.067826,235.456790,1,Toyota Sequoia SUV 2012,121.544348,142.222222,219.046957,186.469136
2,1000669184.jpg,3.072000,50.387302,255.488000,232.431746,1,Toyota Sequoia SUV 2012,129.280000,141.409524,252.416000,182.044444
3,1000800256.jpg,34.816000,17.355932,230.400000,240.813559,1,Toyota Sequoia SUV 2012,132.608000,129.084746,195.584000,223.457627
4,1001324544.jpg,17.600000,39.822222,217.600000,209.066667,1,Toyota Sequoia SUV 2012,117.600000,124.444444,200.000000,169.244444
...,...,...,...,...,...,...,...,...,...,...,...
2345,993853440.jpg,13.500000,30.360176,228.750000,181.411420,1,Suzuki SX4 Sedan 2012,121.125000,105.885798,215.250000,151.051245
2346,994246656.jpg,14.720000,54.186667,216.000000,205.226667,1,Suzuki SX4 Sedan 2012,115.360000,129.706667,201.280000,151.040000
2347,999489536.jpg,20.000000,74.213763,235.000000,218.518302,1,Toyota Sequoia SUV 2012,127.500000,146.366032,215.000000,144.304539
2348,999686144.jpg,32.853333,97.673846,225.280000,233.944615,1,Toyota Sequoia SUV 2012,129.066667,165.809231,192.426667,136.270769


In [9]:
labels_normalized2.index = range(len(labels_normalized2))
labels_normalized2[["x_min", "x_max", "x_mid", "w"]] = labels_normalized2[["x_min", "x_max", "x_mid", "w"]] / w
labels_normalized2[["y_min", "y_max", "y_mid", "h"]] = labels_normalized2[["y_min", "y_max", "y_mid", "h"]] / h
labels_normalized2

,im_name,x_min,y_min,x_max,y_max,class,models,x_mid,y_mid,w,h
0,1000407040.jpg,0.017500,0.104000,0.980000,0.900000,1,Toyota Sequoia SUV 2012,0.498750,0.502000,0.962500,0.796000
1,1000472576.jpg,0.046957,0.191358,0.902609,0.919753,1,Toyota Sequoia SUV 2012,0.474783,0.555556,0.855652,0.728395
2,1000669184.jpg,0.012000,0.196825,0.998000,0.907937,1,Toyota Sequoia SUV 2012,0.505000,0.552381,0.986000,0.711111
3,1000800256.jpg,0.136000,0.067797,0.900000,0.940678,1,Toyota Sequoia SUV 2012,0.518000,0.504237,0.764000,0.872881
4,1001324544.jpg,0.068750,0.155556,0.850000,0.816667,1,Toyota Sequoia SUV 2012,0.459375,0.486111,0.781250,0.661111
...,...,...,...,...,...,...,...,...,...,...,...
2340,993853440.jpg,0.052734,0.118594,0.893555,0.708638,1,Suzuki SX4 Sedan 2012,0.473145,0.413616,0.840820,0.590044
2341,994246656.jpg,0.057500,0.211667,0.843750,0.801667,1,Suzuki SX4 Sedan 2012,0.450625,0.506667,0.786250,0.590000
2342,999489536.jpg,0.078125,0.289898,0.917969,0.853587,1,Toyota Sequoia SUV 2012,0.498047,0.571742,0.839844,0.563690
2343,999686144.jpg,0.128333,0.381538,0.880000,0.913846,1,Toyota Sequoia SUV 2012,0.504167,0.647692,0.751667,0.532308


In [10]:
filenames_val = np.random.choice(labels_normalized2["im_name"], int(0.2 * len(labels_normalized2)))

In [11]:
os.makedirs("../data/data_YOLO4/train/images", exist_ok=True)
os.makedirs("../data/data_YOLO4/train/labels", exist_ok=True)
os.makedirs("../data/data_YOLO4/val/images", exist_ok=True)
os.makedirs("../data/data_YOLO4/val/labels", exist_ok=True)

shutil.rmtree("../data/data_YOLO4/train/images")
shutil.rmtree("../data/data_YOLO4/train/labels")
shutil.rmtree("../data/data_YOLO4/val/images")
shutil.rmtree("../data/data_YOLO4/val/labels")

os.makedirs("../data/data_YOLO4/train/images", exist_ok=True)
os.makedirs("../data/data_YOLO4/train/labels", exist_ok=True)
os.makedirs("../data/data_YOLO4/val/images", exist_ok=True)
os.makedirs("../data/data_YOLO4/val/labels", exist_ok=True)

In [12]:
# for i, filename in enumerate(labels_normalized2["im_name"]):
#     if filename in filenames_val:
#         plt.imsave("../data/data_YOLO2/val/images/" + filename, images_normalized[i])
#         labels_normalized2.iloc[i,[5,1,2,3,4,]].to_csv("../data/data_YOLO2/val/labels/" + filename[:-4] + ".txt", header = False, index = False, lineterminator = " ")
#     else :
#         plt.imsave("../data/data_YOLO2/train/images/" + filename, images_normalized[i])
#         labels_normalized2.iloc[i,[5,1,2,3,4,]].to_csv("../data/data_YOLO2/train/labels/" + filename[:-4] + ".txt", header = False, index = False, lineterminator = " ")

In [13]:
for i, filename in enumerate(labels_normalized2["im_name"]):
    if filename in filenames_val:
        plt.imsave("../data/data_YOLO4/val/images/" + filename, images_normalized[i])
        labels_normalized2.iloc[i,[5,7,8,9,10]].to_csv("../data/data_YOLO4/val/labels/" + filename[:-4] + ".txt", header = False, index = False, lineterminator = " ")
    else :
        plt.imsave("../data/data_YOLO4/train/images/" + filename, images_normalized[i])
        labels_normalized2.iloc[i,[5,7,8,9,10]].to_csv("../data/data_YOLO4/train/labels/" + filename[:-4] + ".txt", header = False, index = False, lineterminator = " ")

In [2]:
import os
filename = os.listdir("../data/data_YOLO3/val/images")

In [3]:
filename

['568393728.jpg',
 '141164544.jpg',
 '50069504.jpg',
 '942211072.jpg',
 '368312320.jpg',
 '441384960.jpg',
 '50331648.jpg',
 '245104640.jpg',
 '902234112.jpg',
 '130154496.jpg',
 '751697920.jpg',
 '601227264.jpg',
 '864813056.jpg',
 '121326593325.jpg',
 '563478528.jpg',
 '57933824.jpg',
 '373620736.jpg',
 '831848448.jpg',
 '39321600.jpg',
 '227803136.jpg',
 '574423040.jpg',
 '157876224.jpg',
 '427687936.jpg',
 '960036864.jpg',
 '601882624.jpg',
 '2717303044849.jpg',
 '445448192.jpg',
 '70189056.jpg',
 '629407744.jpg',
 '92274688.jpg',
 '95944704.jpg',
 '271843328.jpg',
 '973537280.jpg',
 '491913216.jpg',
 '157483008.jpg',
 '367001600.jpg',
 '133234688.jpg',
 '754188288.jpg',
 '370475008.jpg',
 '20152498337.jpg',
 '737607680.jpg',
 '173071853.jpg',
 '373227520.jpg',
 '312737792.jpg',
 '448069632.jpg',
 '60878955456.jpg',
 '999948288.jpg',
 '34668544.jpg',
 '512163840.jpg',
 '319160320.jpg',
 '951713792.jpg',
 '419364864.jpg',
 '602275840.jpg',
 '1009909760.jpg',
 '753729536.jpg',
 '6454